<a href="https://colab.research.google.com/github/Devetec/LinuxComputer/blob/main/Linux_Computer_through_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Linux Computer through Colab
**DO NOT SHARE THIS WITH ANYONE ELSE WITHOUT PERMISSION.**

This utilizes [Chrome Remote Desktop](https://remotedesktop.google.com) and Google Colab to give an online computer experience.

This computer is quickest to set up for running Google Chrome. You can also install programs on this computer, however this might require *a bit* of troubleshooting.

Run the first cell, input the username that you want to use on this temporary computer, same with the password. 

**DO NOT USE YOUR REAL PASSWORD**

The second cell would allow you to have a persistent hard drive, if you will actually use this a computer. Note: programs installed on the computer would not transfer through the hard drive.

The third cell should be kept running to keep Colab awake.

In [ ]:
#@title ***Press Play to Start***
#@markdown  It takes 4-5 minutes for it to get ready.

#@markdown  Visit http://remotedesktop.google.com/headless and copy the command after authentication for the CRP variable.

username = "username" #@param {type:"string"}
password = "password" #@param {type:"string"}
CRP = "DISPLAY= /opt/google/chrome-remote-desktop/start-host --code=\"4/0AY0e-g7Kd8T1i0dcsBviZYp-WitZSrbZVFE0g95jVah1b7HZlxgysu10Xp4S08ZkkuGAWg\" --redirect-url=\"https://remotedesktop.google.com/_/oauthredirect\" --name=$(hostname)" #@param {type:"string"}

# Creation of user
! sudo useradd -m $username &> /dev/null

# Add user to sudo group
! sudo adduser $username sudo &> /dev/null
    
# Set password of user to 'root'
! echo '$username:$password' | sudo chpasswd

# Change default shell from sh to bash
! sed -i 's/\/bin\/sh/\/bin\/bash/g' /etc/passwd

def CRD():
    with open('install.sh', 'w') as script:
        script.write("""#! /bin/bash

b='\033[1m'
r='\E[31m'
g='\E[32m'
c='\E[36m'
endc='\E[0m'
enda='\033[0m'

printf "\n\n$c$b    Loading Installer $endc$enda" >&2
if sudo apt-get update &> /dev/null
then
    printf "\r$g$b    Installer Loaded $endc$enda\n" >&2
else
    printf "\r$r$b    Error Occured $endc$enda\n" >&2
    exit
fi

printf "\n$g$b    Installing Chrome Remote Desktop $endc$enda" >&2
{
    wget https://dl.google.com/linux/direct/chrome-remote-desktop_current_amd64.deb
    sudo dpkg --install chrome-remote-desktop_current_amd64.deb
    sudo apt install --assume-yes --fix-broken
} &> /dev/null &&
printf "\r$c$b    Chrome Remote Desktop Installed $endc$enda\n" >&2 ||
{ printf "\r$r$b    Error Occured $endc$enda\n" >&2; exit; }
sleep 3

printf "$g$b    Installing Desktop Environment $endc$enda" >&2
{
    sudo DEBIAN_FRONTEND=noninteractive \
        apt install --assume-yes xfce4 desktop-base
    sudo bash -c 'echo "exec /etc/X11/Xsession /usr/bin/xfce4-session" > /etc/chrome-remote-desktop-session'  
    sudo apt install --assume-yes xscreensaver
    sudo systemctl disable lightdm.service
} &> /dev/null &&
printf "\r$c$b    Desktop Environment Installed $endc$enda\n" >&2 ||
{ printf "\r$r$b    Error Occured $endc$enda\n" >&2; exit; }
sleep 3

printf "$g$b    Installing Google Chrome $endc$enda" >&2
{
    wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
    sudo dpkg --install google-chrome-stable_current_amd64.deb
    sudo apt install --assume-yes --fix-broken
} &> /dev/null &&
printf "\r$c$b    Google Chrome Installed $endc$enda\n" >&2 ||
printf "\r$r$b    Error Occured $endc$enda\n" >&2
sleep 3

printf "$g$b    Installing other Tools $endc$enda" >&2
if sudo apt install nautilus nano -y &> /dev/null
then
    printf "\r$c$b    Other Tools Installed $endc$enda\n" >&2
else
    printf "\r$r$b    Error Occured $endc$enda\n" >&2
fi
sleep 3

printf "\n$g$b    Installation Completed $endc$enda\n\n" >&2""")

    !chmod +x install.sh
    !./install.sh

    # Adding user to CRP group
    !sudo adduser $username chrome-remote-desktop &> /dev/null

    # Finishing Work
    !su - $username -c """$CRP"""

    print("Finished Succesfully")

try:
    if username:
        if CRP == "" :
            print("Please enter authcode from the given link")
        else:
            CRD()
except NameError:
    print("username variable not found")
    print("Create a User First")



    Installer Loaded 

    Chrome Remote Desktop Installed 
    Desktop Environment Installed 
    Google Chrome Installed 
    Other Tools Installed 

    Installation Completed 

Enter a PIN of at least six digits: 
Enter the same PIN again: 
[0128/152726.804114:ERROR:cert_verify_proc_builtin.cc(559)] No net_fetcher for performing AIA chasing.
[0128/152726.881723:WARNING:host_config.cc(54)] Failed to read /home/devetec/.config/chrome-remote-desktop/host#94d628654f800ddd684326ec6e6f5c5d.json
[0128/152727.073037:ERROR:cert_verify_proc_builtin.cc(559)] No net_fetcher for performing AIA chasing.
[0128/152727.596039:INFO:daemon_controller_delegate_linux.cc(99)] 2021-01-28 15:27:27,583:INFO:User 'devetec' is already a member of 'chrome-remote-desktop'.

[0128/152729.394931:INFO:daemon_controller_delegate_linux.cc(99)] [0128/152727.719659:INFO:remoting_user_session.cc(754)] Daemon process started in the background, logging to '/tmp/chrome_remote_desktop_20210128_152727_e9pSlC'
Using host_

Now that you have run the first cell, you can see this computer under "Remote Devices" in [Chrome Remote Desktop](https://remotedesktop.google.com).

In [ ]:
#@title ***Mount Google Drive***
#@markdown Google Drive can be used as a persistent hard drive. <br>
#@markdown Mounted at the `user` home directory.
def MountGDrive():
    from google.colab import drive

    ! runuser -l $user -c "yes | python3 -m pip install --user google-colab"  > /dev/null 2>&1

    mount = """from os import environ as env
from google.colab import drive

env['CLOUDSDK_CONFIG']  = '/content/.config'
drive.mount('{}')""".format(mountpoint)

    with open('/content/mount.py', 'w') as script:
        script.write(mount)

    ! runuser -l $user -c "python3 /content/mount.py"

try:
    if username:
        mountpoint = "/home/"+username+"/drive"
        user = username
except NameError:
    print("username variable not found, mounting at `/content/drive' using `root'")
    mountpoint = '/content/drive'
    user = 'root'

MountGDrive()

In [ ]:
#@title Run this to keep Colab from disconnecting.

while True:
  i = 1